In [2]:
!pip install apache-airflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 43.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.2/233.2 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.5/44.5 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.9/231.9 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.9/63.9 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.3/102.3 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.7/119.7 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 

In [5]:
from datetime import datetime, timedelta

from airflow import DAG
from airflow.operators.bash import BashOperator

# Define default arguments
default_args = {
    'owner': 'petkao',
    'start_date': datetime.today(),  # Consider using a fixed date like datetime(2025, 5, 6) for stability
    'email': ['petkao@gmail.com'],
    'email_on_failure': True,
    'email_on_retry': True,
    'retries': 1,
    'retry_delay': timedelta(minutes=5)
}

# Define the DAG
with DAG(
    dag_id='ETL_toll_data',
    default_args=default_args,
    description='Apache Airflow Final Assignment',
    # Pass schedule_interval within the schedule argument
    schedule='@daily',
    catchup=False,
    tags=['assignment', 'etl']
) as dag:

    etl_bash_task = BashOperator(
        task_id='run_etl_script',
        bash_command='bash /path/to/etl_toll_data.sh'
    )

    etl_bash_task

<ipython-input-5-e058351aea2b>:4 DeprecationWarning: The `airflow.operators.bash.BashOperator` class is deprecated. Please use `'airflow.providers.standard.operators.bash.BashOperator'`.

In [17]:
from datetime import datetime, timedelta

from airflow import DAG
from airflow.operators.bash import BashOperator

# Define default arguments
default_args = {
    'owner': 'petkao',
    'start_date': datetime.today(),
    'email': ['petkao@gmail.com'],
    'email_on_failure': True,
    'email_on_retry': True,
    'retries': 1,
    'retry_delay': timedelta(minutes=5)
}

# Define the DAG
with DAG(
    dag_id='ETL_toll_data',
    default_args=default_args,
    description='Apache Airflow Final Assignment',
    # Pass schedule_interval value to the schedule argument
    schedule='@daily',
    catchup=False,
    tags=['assignment', 'etl']
) as dag:

    # Task to unzip the toll data archive
    unzip_data = BashOperator(
        task_id='unzip_data',
        bash_command='tar -xvzf /home/project/airflow/dags/finalassignment/tolldata.tgz -C /home/project/airflow/dags/finalassignment/'
    )

    extract_data_from_csv = BashOperator(
        task_id='extract_data_from_csv',
        bash_command="""
        cut -d',' -f1,2,3,4 /home/project/airflow/dags/finalassignment/vehicle-data.csv > /home/project/airflow/dags/finalassignment/csv_data.csv
        """
    )

    extract_data_from_tsv = BashOperator(
        task_id='extract_data_from_tsv',
        bash_command="""
        cut -f5,6,7 --output-delimiter=',' /home/project/airflow/dags/finalassignment/tollplaza-data.tsv > /home/project/airflow/dags/finalassignment/tsv_data.csv
        """
    )

    extract_data_from_fixed_width = BashOperator(
        task_id='extract_data_from_fixed_width',
        bash_command="""
        cut -c40-49,50-59 /home/project/airflow/dags/finalassignment/payment-data.txt | \
        sed 's/ \{1,\}/,/g' > /home/project/airflow/dags/finalassignment/fixed_width_data.csv
        """
    )

    consolidate_data = BashOperator(
        task_id='consolidate_data',
        bash_command="""
        paste -d',' \
        /home/project/airflow/dags/finalassignment/csv_data.csv \
        /home/project/airflow/dags/finalassignment/tsv_data.csv \
        /home/project/airflow/dags/finalassignment/fixed_width_data.csv \
        > /home/project/airflow/dags/finalassignment/extracted_data.csv
        """
    )

    unzip_data >> extract_data_from_csv >> extract_data_from_tsv >> extract_data_from_fixed_width >> consolidate_data

    transform_data = BashOperator(
        task_id='transform_data',
        bash_command="""
        awk -F',' 'BEGIN {OFS=","} { $4=toupper($4); print }' \
        /home/project/airflow/dags/finalassignment/extracted_data.csv \
        > /home/project/airflow/dags/finalassignment/staging/transformed_data.csv
        """
    )

    unzip_data >> extract_data_from_csv >> extract_data_from_tsv >> extract_data_from_fixed_width >> consolidate_data >> transform_data



<ipython-input-17-ae4ffa6f0564>:4 DeprecationWarning: The `airflow.operators.bash.BashOperator` class is deprecated. Please use `'airflow.providers.standard.operators.bash.BashOperator'`.